In [1]:
from nltk.stem.isri import ISRIStemmer
from camel_tools.utils.normalize import normalize_alef_maksura_ar, normalize_teh_marbuta_ar, normalize_alef_ar
import pandas as pd
import re

# Read the lexicons

In [17]:
plus_russia_lexicon = pd.read_csv("../csvs/russia.csv", header=None)
plus_ukraine_lexicon = pd.read_csv("../csvs/ukraine.csv", header=None)
plus_west_lexicon = pd.read_csv("../csvs/west_america.csv", header=None)

In [19]:
plus_russia_lexicon.head

<bound method NDFrame.head of             0     1
0        هجوم  3.00
1         غزو  7.25
2         فشل  6.00
3       ادعاء  7.00
4       جريمة  9.50
..        ...   ...
195     سيطره -9.50
196  بالسيطره -9.50
197   للسيطره -9.50
198     تسيطر -9.50
199    وتسيطر -9.50

[200 rows x 2 columns]>

In [38]:
def make_dict(lexicon):
    lexicon_dict = {}
    for _, row in lexicon.iterrows():
        word = row[0]
        score = row[1]
        lexicon_dict[word] = score
    return lexicon_dict

In [40]:
plus_rus_dic = make_dict(plus_russia_lexicon)
plus_ukr_dic = make_dict(plus_ukraine_lexicon)
plus_west_dic = make_dict(plus_west_lexicon)

# Complete the phrase and add it to the full dict

In [48]:
complete_lexicon = {}

In [49]:
def complete_phrase(lexicon_dict, words_to_add):
    for word, score in lexicon_dict.items():
        phrases = []
        for word_to_add in words_to_add:
            phrase = word + " " + word_to_add
            phrases.append(phrase)        
        
        for phrase in phrases:
            complete_lexicon[phrase] = score
    

In [50]:
complete_phrase(plus_rus_dic, ["روسيا", "الروسية", "الروسي" ,'روسيا', 'الروسي'])
complete_phrase(plus_ukr_dic, ["أوكرانيا", "الأوكرانية", "الأوكراني" ,'أوكرانيا', 'الأوكراني'])
complete_phrase(plus_west_dic, ["أمريكا", "الأمريكية", "الأمريكي" ,'أمريكا', 'الأمريكي', 'الغرب', 'الغربية', 'الغربي'])

In [51]:
complete_lexicon

{'هجوم روسيا': 3.0,
 'هجوم الروسية': 3.0,
 'هجوم الروسي': 3.0,
 'غزو روسيا': 7.25,
 'غزو الروسية': 7.25,
 'غزو الروسي': 7.25,
 'فشل روسيا': 6.0,
 'فشل الروسية': 6.0,
 'فشل الروسي': 6.0,
 'ادعاء روسيا': 7.0,
 'ادعاء الروسية': 7.0,
 'ادعاء الروسي': 7.0,
 'جريمة روسيا': 9.5,
 'جريمة الروسية': 9.5,
 'جريمة الروسي': 9.5,
 'جرم روسيا': 5.75,
 'جرم الروسية': 5.75,
 'جرم الروسي': 5.75,
 'جرائم روسيا': 4.25,
 'جرائم الروسية': 4.25,
 'جرائم الروسي': 4.25,
 'سقوط روسيا': 8.75,
 'سقوط الروسية': 8.75,
 'سقوط الروسي': 8.75,
 'خسائر روسيا': 5.25,
 'خسائر الروسية': 5.25,
 'خسائر الروسي': 5.25,
 'وحشية روسيا': 4.75,
 'وحشية الروسية': 4.75,
 'وحشية الروسي': 4.75,
 'فقدت روسيا': 9.25,
 'فقدت الروسية': 9.25,
 'فقدت الروسي': 9.25,
 'الهجوم روسيا': 7.75,
 'الهجوم الروسية': 7.75,
 'الهجوم الروسي': 7.75,
 'الغزو روسيا': 5.25,
 'الغزو الروسية': 5.25,
 'الغزو الروسي': 5.25,
 'الفشل روسيا': 9.75,
 'الفشل الروسية': 9.75,
 'الفشل الروسي': 9.75,
 'الادعاء روسيا': 9.0,
 'الادعاء الروسية': 9.0,
 'الادعاء الروسي': 9.0

In [55]:
phrases_lexicon = pd.read_csv("../csvs/phrases.csv")

In [56]:
phrases_lexicon

,string,number
0,حصار-قصف كييف/ماريوبول/خاركيف,5.50
1,بوتين مجرم/قاتل,9.75
2,عقوبات اقتصادية/سياسية,5.50
3,حقوق الانسان,5.75
4,الردع الشعبي,8.25
...,...,...
87,روسيا كسبانة/فائزة,-7.25
88,الغرب/امريكا عنصري,-8.25
89,الغرب يدعي/امريكا تدعي,-7.50
90,الغرب/امريكا منافق,-8.50


In [67]:
phrases_dict = make_dict(phrases_lexicon)

In [2]:
def remove_last_word(phrase):
    words = phrase.split()
    word = words.pop()
    return " ".join(words), word

def remove_first_word(phrase):
    words = phrase.split()
    word = words.pop(0)
    return " ".join(words), word 

def get_parts(phrase, slash_position):
    first_part = phrase[:slash_position]
    first_part, word1= remove_last_word(first_part)
    second_part = phrase[slash_position+1:]
    second_part, word2 = remove_first_word(second_part)
    
    options = word1.split('/') + word2.split('/')
    return first_part, second_part, options

def expand_strings(strings: dict):
    expanded_strings = {}
    for string, score in strings.items():
        # detect the slash position
        slash_position = string.find("/")
        
        if slash_position == -1:
            expanded_strings[string] = score
            continue
            
        first_part, second_part, options = get_parts(string, slash_position)
        
        for option in options:
            expanded_string = first_part + " " + option + " " + second_part
            expanded_strings[expanded_string.strip()] = score
        
    return expanded_strings

In [76]:
final_phrases = expand_strings(phrases_dict)
final_phrases

{'حصار-قصف كييف': 5.5,
 'حصار-قصف ماريوبول': 5.5,
 'حصار-قصف خاركيف': 5.5,
 'بوتين مجرم': 9.75,
 'بوتين قاتل': 9.75,
 'عقوبات اقتصادية': 5.5,
 'عقوبات سياسية': 5.5,
 'حقوق الانسان': 5.75,
 'الردع الشعبي': 8.25,
 'الهجوم البربري': 8.5,
 'الهجوم العنيف': 8.5,
 'مساعدات': 5.0,
 'اجلاء': 3.0,
 'اخلاء': 3.0,
 'الغزاة الروس': 8.0,
 'رفض الحرب': 7.5,
 'دماء المدنيين': 7.75,
 'عمل اجرامي': 7.25,
 'مجزرة': 7.75,
 'اوكرانيا تدافع': 7.0,
 'زيلنسكي بطل': 9.0,
 'زيلنسكي عظيم': 9.0,
 'القانون الدولي': 3.25,
 'مظاهرات': 3.5,
 'احتلت': 5.0,
 'ضد هجوم': 4.5,
 'نحذر': 3.0,
 'الغاشم': 4.5,
 'اباده جماعيه': 6.25,
 'دوله منبوذه': 4.0,
 'حرب العصابات': 4.5,
 'سوريا': 4.5,
 'وزير خارجيه اوكرانيا': 2.5,
 'اوكرانيا تنزف': 7.25,
 'العواقب الاقتصاديه': 4.25,
 'روسيا معتدية': 8.25,
 'روسيا معتدين': 8.25,
 'اوكرانيا تقاوم': 5.75,
 'جرائم حرب': 6.5,
 'جريمة حرب': 6.5,
 'لعن الله روسيا': 8.5,
 'لعن الله بوتين': 8.5,
 'تهديد لاوكرانيا': 5.5,
 'ارهاب لاوكرانيا': 5.5,
 'روسيا تهجم': 3.75,
 'روسيا تغزو': 7.25,
 'روسيا ت

In [77]:
strings = [['النازيون الجدد', 'النازيين الجدد', 'النازية الجديدة'], ['الجرائم الاوكرانية', 'الجريمة الاوكرانية', 'الجرائم النازية', 'الجريمة النازية'], ['الغرب يدعي', 'امريكا تدعي'], ['قصف كييف', 'حصار كييف', 'قصف ماريوبول', 'حصار ماريوبول', 'حصار خاركيف', 'قصف خاركيف']]
scores = [-5.5, -9.5, -7.5, 5.5] 
to_remove = ['الجريمة الاوكرانية/النازية', 'الجرائم الاوكرانية/النازية', 'النازيين الجدد - النازية الجديدة', 
             'النازيون الجدد - النازية الجديدة', 'الغرب امريكا تدعي','الغرب يدعي تدعي', 'حصار-قصف خاركيف', 'حصار-قصف ماريوبول', 'حصار-قصف كييف']
for words, score in zip(strings, scores):
    for word in words:
        final_phrases[word] = score 
        
for word in to_remove:
    del final_phrases[word]

In [ ]:
final_phrases

In [ ]:
full_lexicon = {**complete_lexicon, **final_phrases}

In [ ]:
len(full_lexicon)

In [ ]:
full_lexicon

In [ ]:
pd.DataFrame(full_lexicon.items()).to_csv("full_lexicon.csv", index=False)

# Data labeling

In [3]:
# df = pd.read_csv("sentiments2.csv")
# sentences = df["preprocessed_tweet_text"].astype(str)

stemmer = ISRIStemmer()

In [33]:
def clean_str(tweet):
    # print(tweet)
    if type(tweet) is not str:
        return ""
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    tweet = re.sub(r'@[^\s]+', '', tweet)

    # Removing links
    tweet = re.sub(r'(https?://\S+)|(www\.\S+)', '', tweet)

    # Removing emojis
    tweet = re.sub(r'[^\u0600-\u06FF\s]', '', tweet)  # Preserve Arabic characters

    # Removing hashtags' octothorpe, underscores, and hyphens
    tweet = re.sub(r'[\#\_\-\u061f]', '', tweet)


    # Normalizing different forms of letters in Arabic
    tweet = normalize_alef_maksura_ar(tweet)
    tweet = normalize_alef_ar(tweet)
    tweet = normalize_teh_marbuta_ar(tweet)
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", tweet)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    words = text.split()
    for i in range(len(words)):
        words[i] = stemmer.norm(words[i], num=1)
        if not words[i] in stemmer.stop_words:    # exclude stop words from being processed
            words[i] = stemmer.pre32(words[i])        # remove length three and length two prefixes in this order
            words[i] = stemmer.suf32(words[i])        # remove length three and length two suffixes in this order
            words[i] = stemmer.waw(words[i])          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            words[i] = stemmer.norm(words[i], num=2)
        else :
            words[i] = " "
        
       
    text = " ".join(words)
    if type(text) is not str:
        print(text)
        return ""
    return text

In [85]:
sentences

0         هل كانت زياره البابا الي بودابست لعمل وساطه بي...
1         عاجل \n\nاطلقت القوات الروسيه صواريخ كروز من ب...
2         حتي الدرونات الايرانيه خلاصوا  \nحتي وان عانت ...
3         هزت انفجارات ضخمه متعدده مدينه بافلوهراد في من...
4         اوكرانيا تطالب الناتو بمزيد من الاسلحه والمعدا...
                                ...                        
162401    مقطع يوثق استخدام الجنود الاوكرانيين للشقق الس...
162402    هههههه طيب ليه ما استخدمته امريكا في افغانستان...
162403    رئيس هيئه الاركان المشتركه الامريكيه روسيا  فش...
162404    فرصه ذهبيه امام الاخوان لتصحيح علاقتهم مع اوكر...
162405        ستخسر روسيا هذه الحرب ،خذها مني علي محمل الجد
Name: preprocessed_tweet_text, Length: 162406, dtype: object

In [87]:
len(sentences)

162406

In [88]:
cleaned_lexicon = {}
for phrase, score in full_lexicon.items():
    cleaned_lexicon[clean_str(phrase)] = score


In [89]:
cleaned_lexicon

{'هجوم روس': 7.75,
 'هجوم روسيه': 7.75,
 'هجوم روسي': 7.75,
 'غزو روس': 5.25,
 'غزو روسيه': 5.25,
 'غزو روسي': 5.25,
 'فشل روس': 9.75,
 'فشل روسيه': 9.75,
 'فشل روسي': 9.75,
 'ادعاء روس': 9.0,
 'ادعاء روسيه': 9.0,
 'ادعاء روسي': 9.0,
 'جريمه روس': 4.25,
 'جريمه روسيه': 4.25,
 'جريمه روسي': 4.25,
 'جرم روس': 5.75,
 'جرم روسيه': 5.75,
 'جرم روسي': 5.75,
 'جرائم روس': 9.5,
 'جرائم روسيه': 9.5,
 'جرائم روسي': 9.5,
 'سقوط روس': 8.75,
 'سقوط روسيه': 8.75,
 'سقوط روسي': 8.75,
 'خسائر روس': 5.25,
 'خسائر روسيه': 5.25,
 'خسائر روسي': 5.25,
 'وحشيه روس': 4.75,
 'وحشيه روسيه': 4.75,
 'وحشيه روسي': 4.75,
 'فقدت روس': 9.25,
 'فقدت روسيه': 9.25,
 'فقدت روسي': 9.25,
 'ارهاب روس': 5.5,
 'ارهاب روسيه': 5.5,
 'ارهاب روسي': 5.5,
 'احتلال روس': 8.5,
 'احتلال روسيه': 8.5,
 'احتلال روسي': 8.5,
 'قصف روس': 2.25,
 'قصف روسيه': 2.25,
 'قصف روسي': 2.25,
 'عدو روس': 8.5,
 'عدو روسيه': 8.5,
 'عدو روسي': 8.5,
 'عمليه روس': 4.75,
 'عمليه روسيه': 4.75,
 'عمليه روسي': 4.75,
 'اعتداء روس': 9.25,
 'اعتداء روسيه': 9.25,

In [91]:
pd.DataFrame(cleaned_lexicon.items()).to_csv("cleaned_lexicon.csv", index=False)

In [90]:
phrases_lens_counts = {}
for phrase in cleaned_lexicon.keys():
    phrase_len = len(phrase.split())
    if phrase_len in phrases_lens_counts:
        phrases_lens_counts[phrase_len] += 1
    else:
        phrases_lens_counts[phrase_len] = 1
phrases_lens_counts

{2: 892, 1: 19, 3: 12}

In [94]:
# add a score column to the dataframe
df["partiality score"] = 0

In [100]:
df.drop('score', axis=1, inplace=True)

In [121]:
df.dropna(subset=["preprocessed_tweet_text"], inplace=True)

In [101]:
df

,tweet_id,preprocessed_tweet_text,Sentiments,partiality score
0,1652825135068463112,هل كانت زياره البابا الي بودابست لعمل وساطه بي...,neutral,0
1,1652825064297865220,عاجل \n\nاطلقت القوات الروسيه صواريخ كروز من ب...,neutral,0
2,1652825038549123072,حتي الدرونات الايرانيه خلاصوا \nحتي وان عانت ...,negative,0
3,1652825015383891968,هزت انفجارات ضخمه متعدده مدينه بافلوهراد في من...,neutral,0
4,1652825012036943873,اوكرانيا تطالب الناتو بمزيد من الاسلحه والمعدا...,neutral,0
...,...,...,...,...
162401,1641953773684678656,مقطع يوثق استخدام الجنود الاوكرانيين للشقق الس...,negative,0
162402,1641953678964801537,هههههه طيب ليه ما استخدمته امريكا في افغانستان...,negative,0
162403,1641953645511028737,رئيس هيئه الاركان المشتركه الامريكيه روسيا فش...,neutral,0
162404,1641953600627781638,فرصه ذهبيه امام الاخوان لتصحيح علاقتهم مع اوكر...,positive,0


In [9]:
def score_bigrams(tweet):
    words = tweet.split()
    score = 0
    counts = 0
    for i in range(len(words)-1):
        bigram = words[i] + " " + words[i+1]
        if bigram in cleaned_lexicon:
            score += cleaned_lexicon[bigram]
            counts += 1
    return score, counts

def score_trigrams(tweet):
    words = tweet.split()
    score = 0
    counts = 0
    for i in range(len(words)-2):
        trigram = words[i] + " " + words[i+1] + " " + words[i+2]
        if trigram in cleaned_lexicon:
            score += cleaned_lexicon[trigram]
            counts += 1
    return score, counts

def score_unigrams(tweet):
    words = tweet.split()
    score = 0
    counts = 0
    for word in words:
        if word in cleaned_lexicon:
            score += cleaned_lexicon[word]
            counts += 1
    return score, counts

def score_tweet(tweet):
    return tuple(map(lambda i, j, k: i + j + k, score_unigrams(tweet), score_bigrams(tweet), score_trigrams(tweet)))

In [35]:
def score_row(row, text_column="preprocessed_tweet_text"):
    if type(row[text_column]) is not str or row[text_column] == "":
        return 0, 0
        
    return score_tweet(row[text_column])
    

In [132]:
df['partiality score'] = df.apply(lambda row: score_row(row), axis=1)

In [133]:
df

,tweet_id,preprocessed_tweet_text,Sentiments,partiality score
0,1652825135068463112,هل كانت زياره البابا الي بودابست لعمل وساطه بي...,neutral,0.0
1,1652825064297865220,عاجل \n\nاطلقت القوات الروسيه صواريخ كروز من ب...,neutral,0.0
2,1652825038549123072,حتي الدرونات الايرانيه خلاصوا \nحتي وان عانت ...,negative,0.0
3,1652825015383891968,هزت انفجارات ضخمه متعدده مدينه بافلوهراد في من...,neutral,-5.0
4,1652825012036943873,اوكرانيا تطالب الناتو بمزيد من الاسلحه والمعدا...,neutral,0.0
...,...,...,...,...
162401,1641953773684678656,مقطع يوثق استخدام الجنود الاوكرانيين للشقق الس...,negative,0.0
162402,1641953678964801537,هههههه طيب ليه ما استخدمته امريكا في افغانستان...,negative,0.0
162403,1641953645511028737,رئيس هيئه الاركان المشتركه الامريكيه روسيا فش...,neutral,0.0
162404,1641953600627781638,فرصه ذهبيه امام الاخوان لتصحيح علاقتهم مع اوكر...,positive,0.0


In [134]:
# get the rows with non zero partiality scores
scored_df = df[df["partiality score"] != 0]

In [135]:
scored_df

,tweet_id,preprocessed_tweet_text,Sentiments,partiality score
3,1652825015383891968,هزت انفجارات ضخمه متعدده مدينه بافلوهراد في من...,neutral,-5.00
15,1652824086970527744,في اوكرانيا اباده جماعيه للروس الاوكراينيين يغ...,negative,6.25
16,1652824036638851076,تقارير تفيد بان اكثر من قاذفه قنابل روسيه من ...,neutral,8.50
25,1652823394503499778,اوكرانيا\n\nقصف روسي جنوني يطال مستودع ضخم للس...,negative,2.25
26,1652823389331914753,وش جبل صين وش حجار البيت وش روسيا التتار وش ج...,negative,-5.25
...,...,...,...,...
162264,1641963118468165635,اصابه موفقه \nتم سحق السريه بالكامل \n\nروسيا...,neutral,-10.00
162267,1641962905770835969,لا تكون اناني وتحب نفسك وتكره دوله وتحب الغرب ...,negative,6.50
162286,1641961580819537920,رئيس هيئه الاركان المشتركه الامريكيه مارك ميلي...,neutral,-5.00
162314,1641960139744858113,الامم المتحده لا تجد اي مظاهر للاباده الجماعيه...,negative,3.50


In [136]:
len(scored_df)

3406

# label the whole dataset

In [7]:
cleaned_lexicon_df = pd.read_csv("cleaned_lexicon.csv")
cleaned_lexicon = {}
# create a dictionary from the lexicon
for _, row in cleaned_lexicon_df.iterrows():
    cleaned_lexicon[row[0]] = row[1]

cleaned_lexicon

{'هجوم روس': 7.75,
 'هجوم روسيه': 7.75,
 'هجوم روسي': 7.75,
 'غزو روس': 5.25,
 'غزو روسيه': 5.25,
 'غزو روسي': 5.25,
 'فشل روس': 9.75,
 'فشل روسيه': 9.75,
 'فشل روسي': 9.75,
 'ادعاء روس': 9.0,
 'ادعاء روسيه': 9.0,
 'ادعاء روسي': 9.0,
 'جريمه روس': 4.25,
 'جريمه روسيه': 4.25,
 'جريمه روسي': 4.25,
 'جرم روس': 5.75,
 'جرم روسيه': 5.75,
 'جرم روسي': 5.75,
 'جرائم روس': 9.5,
 'جرائم روسيه': 9.5,
 'جرائم روسي': 9.5,
 'سقوط روس': 8.75,
 'سقوط روسيه': 8.75,
 'سقوط روسي': 8.75,
 'خسائر روس': 5.25,
 'خسائر روسيه': 5.25,
 'خسائر روسي': 5.25,
 'وحشيه روس': 4.75,
 'وحشيه روسيه': 4.75,
 'وحشيه روسي': 4.75,
 'فقدت روس': 9.25,
 'فقدت روسيه': 9.25,
 'فقدت روسي': 9.25,
 'ارهاب روس': 5.5,
 'ارهاب روسيه': 5.5,
 'ارهاب روسي': 5.5,
 'احتلال روس': 8.5,
 'احتلال روسيه': 8.5,
 'احتلال روسي': 8.5,
 'قصف روس': 2.25,
 'قصف روسيه': 2.25,
 'قصف روسي': 2.25,
 'عدو روس': 8.5,
 'عدو روسيه': 8.5,
 'عدو روسي': 8.5,
 'عمليه روس': 4.75,
 'عمليه روسيه': 4.75,
 'عمليه روسي': 4.75,
 'اعتداء روس': 9.25,
 'اعتداء روسيه': 9.25,

In [8]:
import os

data_path = "../data/russia_ukraine_war_using_academic_api"
new_data_path = "../data/with_scores"
files = [file for file in os.listdir(data_path) if file.endswith(".csv")]
files

['tweets_mar_2023.csv',
 'tweets_jan_2022.csv',
 'tweets_may_2022.csv',
 'tweets_may_2023.csv',
 'tweets_jul_2022.csv',
 'tweets_jun_2022.csv',
 'tweets_feb_2022.csv',
 'tweets_mar_2022.csv',
 'tweets_oct_2022.csv',
 'tweets_jan_2023.csv',
 'tweets_sep_2022.csv',
 'tweets_nov_2022.csv',
 'tweets_feb_2023.csv',
 'tweets_aug_2022.csv',
 'tweets_nov_2021.csv',
 'tweets_dec_2021.csv',
 'tweets_apr_2022.csv',
 'tweets_apr_2023.csv',
 'tweets_dec_2022.csv']

In [ ]:
import pandas as pd
import tqdm
tqdm.tqdm.pandas()

# Create an empty list to store DataFrames
dfs = []
cols = ['tweet_id', 'tweet_text']
s = 8
for i, file in enumerate(files[s:]):
    print(f"Processing file {i+1+s}/{len(files)}")
    current_df = pd.read_csv(os.path.join(data_path, file), usecols=cols)
    current_df['tweet_text'] = current_df['tweet_text'].progress_apply(lambda tweet: clean_str(tweet))
    current_df['partiality_score'] = 0.0
    current_df['n-gram counts'] = 0
    current_df[['partiality_score', 'n-gram counts']] = current_df.progress_apply(lambda row: score_row(row, text_column='tweet_text'), axis=1, result_type='expand')
    
    current_df.to_csv(os.path.join(new_data_path, file), index=False)
    
    del current_df
    print('-' * 50)
    

Processing file 9/19


/tmp/ipykernel_8222/2380929000.py:11: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(os.path.join(data_path, file), usecols=cols)
100%|██████████| 434241/434241 [00:26<00:00, 16164.90it/s]


--------------------------------------------------
Processing file 10/19


/tmp/ipykernel_8222/2380929000.py:11: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(os.path.join(data_path, file), usecols=cols)
  0%|          | 0/315441 [00:00<?, ?it/s]

In [50]:
len(full_df)

219406

In [10]:
len(files)

19

In [51]:
full_df

,tweet_id,tweet_text,partiality_score
0,1641953510903234562,روس تحتج علي تصرف استفزازيه قوات امريكيه في سور,4.50
2,1641953254094381057,روس تحتج علي استفزاز قوات اميركيه في سور,4.50
3,1641953209810862080,توضيح لن يحدث شي مهم هذا اسبوع ومن متوقع تبدا ...,-5.25
5,1641953161295347716,فرنسا بتشوف ايام شرق اوسطيه افريقيه زي الفل يع...,-5.25
7,1641952955136974848,روس تحتج علي استفزاز قوات اميركيه في سور سكاي ...,4.50
...,...,...,...
189657,1542660991330263040,عند تتحدث عن فلسط سيادت وتدافع عن نبي والله ين...,-8.00
189659,1542660359773093888,لن تنسحب روس من سور وتسلم مواقع ل اير الا اذا ...,4.50
189660,1542660179292045312,حربالعالميهالثالثه قرعت طبول روسياواوكران كوري...,-5.25
189667,1542659520056418304,هذا موسم رغم جفاف وقله اسمده ورداءه مبيد كان ا...,-4.00


In [52]:
cleaned_lexicon

{'هجوم روس': 7.75,
 'هجوم روسيه': 7.75,
 'هجوم روسي': 7.75,
 'غزو روس': 5.25,
 'غزو روسيه': 5.25,
 'غزو روسي': 5.25,
 'فشل روس': 9.75,
 'فشل روسيه': 9.75,
 'فشل روسي': 9.75,
 'ادعاء روس': 9.0,
 'ادعاء روسيه': 9.0,
 'ادعاء روسي': 9.0,
 'جريمه روس': 4.25,
 'جريمه روسيه': 4.25,
 'جريمه روسي': 4.25,
 'جرم روس': 5.75,
 'جرم روسيه': 5.75,
 'جرم روسي': 5.75,
 'جرائم روس': 9.5,
 'جرائم روسيه': 9.5,
 'جرائم روسي': 9.5,
 'سقوط روس': 8.75,
 'سقوط روسيه': 8.75,
 'سقوط روسي': 8.75,
 'خسائر روس': 5.25,
 'خسائر روسيه': 5.25,
 'خسائر روسي': 5.25,
 'وحشيه روس': 4.75,
 'وحشيه روسيه': 4.75,
 'وحشيه روسي': 4.75,
 'فقدت روس': 9.25,
 'فقدت روسيه': 9.25,
 'فقدت روسي': 9.25,
 'ارهاب روس': 5.5,
 'ارهاب روسيه': 5.5,
 'ارهاب روسي': 5.5,
 'احتلال روس': 8.5,
 'احتلال روسيه': 8.5,
 'احتلال روسي': 8.5,
 'قصف روس': 2.25,
 'قصف روسيه': 2.25,
 'قصف روسي': 2.25,
 'عدو روس': 8.5,
 'عدو روسيه': 8.5,
 'عدو روسي': 8.5,
 'عمليه روس': 4.75,
 'عمليه روسيه': 4.75,
 'عمليه روسي': 4.75,
 'اعتداء روس': 9.25,
 'اعتداء روسيه': 9.25,

In [53]:
full_df.to_csv("labeled_tweets.csv", index=False)